# Kickstarting the Martian Greenhouse Effect
## A Final Project for Physics 361 by Morgan Baxter and Alex Brovender

For a variety of reasons, including proximity to the sun and atmospheric and regolith composition, Mars is the number one candidate body in the solar system for future human colonization. Despite this status, there are many hurdles which humans will need to overcome in order to permanantly colonize Mars. One of the key problems is the extreme temperature variation. In order to increase the thermal inertia of the atmosphere and generally increase the average temperature, we propose a model where a large amount of CO2 is released suddenly into the atmosphere, with the intention of increasing the temperature at the poles. The Martian ice caps have a considerable amount of CO2 that, if liberated, is likely to further increase this greenhouse effect, and therefore continue to warm the planet. We use simple climate models to find the amount of energy that would be required to liberate enough CO2 to kickstart this greenhouse effect.

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/6/6a/PIA22546-Mars-AnnualCO2ice-N%26SPoles-20180806.gif" width="500"/>
</div>
$\textbf{Figure 1:}$ Extents of north (left) and south (right) polar CO2 ice during a martian year

$\textbf{Source:}$ NASA/JPL-caltech




## Hogg 2008: Glacial cycles and carbon dioxide: A conceptual model

We utilize Hogg, 2008, as a starting point for our model for the Mars climate system. The equations as he provides them are as follows:

$$ c\frac{dT}{dt} = \bar{S} + \sum_i{sin \left (\frac{2 \pi t}{\Gamma_i}\right )} + \bar{G} + A \ln \left(\frac{C}{C_0}\right) - \sigma T^4$$

$$ \frac{dC}{dt} = V - (W_0 + W_1C) + \beta (C_{max} - C)\max\left(\frac{dT}{dt} - \epsilon , 0 \right) $$

These equations in their current form are not useful to us for a variety of reasons. They include terms that account for volcanic activity, the carbon sink of the oceans, orbital pertubations, and the heat capacity of the oceans. As a result, the coupled differential equations that we will use look slightly different. However, they do take a variety of the the phenomena we want to observe into account, so they are a good starting point.

## Modified Equations

Our reduced equations look as follows:

$$ \frac{dT}{dt} = \Big[ \bar{S} + \bar{G} + A \ln \left(\frac{C}{C_0}\right) - \sigma T^4 \Big] /c$$

$$\frac{dC}{dt} = (T^* - 120)\beta$$

Our state variables are $T$, the average surface temperature in Kelvin, and $C$, the mass of CO2 in the Martian atmosphere, in kilograms. Therefore, our parameters must match these units, and we list them below:

$\beta$ is the rate of carbon effect on temperature. It has units of $kg \cdot s^{-1} K^{-1}$.

$T^*$ is the temperature $\textbf{at the poles}$ in Kelvin. This will vary by a set offset $\textbf{NOT YET DETERMINED}$ that will solve issues relating to the average temperature on Mars being more than warm enough to sublimate CO2 at the given pressure.

$\bar{S}$ is the rate of incoming solar radiation. This value is $589 W \cdot m^{-2}$

$\bar{G}$ is the greenhouse warming term. This should initially be on the order of $1 W \cdot m^{-2}$

$A$ represents the effect of CO2 on the radiation budget. It should be set to $5.35 W \cdot m^{-2}$. Might want to play around with this value. Hogg cites a paper that seems to detail where this value comes from.

$\frac{C}{C_0}$ represents the ratio between the mass of the CO2 currently in the atmosphere compared to the initial value. It is dimensionless.

$\sigma$ is the Stefan-Boltzmann constant, which is equal to $\frac{2 \pi^5 k^4}{15c^2 h^3} = 5.67 *10^{-8} W m^{-2} K^{-4}$.

$c$ is the specific heat capacity of CO2. We will take this to be approximately $700 J \cdot kg^{-1} \cdot K^{-1}$. This is a function of temperature and pressure, and therefore it could be more accurate to have this in function form rather than as a constant.

These two equations are enough for us to solve the system.

In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integ

# this function defines the system of equations that we wrote above.
def RHS(w,t,p):
        
    # w holds C and T, the mass of atmospheric carbon and average surface temperature
    C, T = w
    
    # the variable p holds the parameters
    
    SB_constant, solar_constant, G, A, C_0, beta = p
    
    dT = (solar_constant + G + A*np.ln(C/C_0) - (SB_constant*(T**4)))/700
    
    dC = (T - 120)*beta
    
    f = [dT, dC]
    
    return f


In [26]:
#solve = integ.solve_ivp(RHS,,[0.95])


# here we define our constants and put them in a list to hand to the RHS function
SB_constant = 5.67e-8
solar_constant = 589
G = 1
A = 5.35
# just to make euler's method work, will change later, but not a bad value, guessed at initial T and C values, ADJUST LATER
T_0 = 215      
C_0 = 10000
beta = 0.1
parameters = [SB_constant, solar_constant, G, A, C_0, beta]


Now we have our function for the system of differential equations, a list containing our parameters, and initial guesses for starting temperature and atmospheric carbon content. From here, we use the Euler method to solve this system.

In [48]:
# Euler's method for T, stepsize t:
dT = (solar_constant + G + A*np.log(C/C_0) - (SB_constant*(T**4)))/700
dC = (T - 120)*beta
t = 0.0
dt = 1.0
T = T_0
C = C_0

for i in range(100):
    C = C + dC*dt
    T = T + dT*dt
    dT = (solar_constant + G + (A*np.log(C/C_0)) - (SB_constant*(T**4)))/700
    dC = (T - 120)*beta
    t = t + dt
    print(t,"       ",C,"          ",T, "  ", dT, "   ", dC)

1.0         10014.9608571            215.41577824    0.668449059565     9.54157782404
2.0         10024.5024349            216.0842273    0.666281299946     9.60842272999
3.0         10034.1108576            216.7505086    0.664100457425     9.67505085999
4.0         10043.7859085            217.414609057    0.661906648086     9.74146090573
5.0         10053.5273694            218.076515705    0.65969999062     9.80765157054
6.0         10063.335021            218.736215696    0.6574806063     9.8736215696
7.0         10073.2086425            219.393696302    0.655248618959     9.93936963023
8.0         10083.1480122            220.048944921    0.653004154965     10.0048944921
9.0         10093.1529067            220.701949076    0.650747343191     10.0701949076
10.0         10103.2231016            221.352696419    0.648478314993     10.1352696419
11.0         10113.3583712            222.001174734    0.646197204178     10.2001174734
12.0         10123.5584887            222.647371939